In [1]:
import spacy
import re
import textwrap
nlp = spacy.load("es_core_news_sm")

In [4]:
file_path = "../data/emos-vs-punks.txt"

replace_subs = [
    ("\[[^\[\]]+\]: ", ""),
    ("\(SOUNDBITE ARCHIVO\)", ""),
    ("…", "."),
    ("\s+", " ")
]

punctuation_marks = ["»", "“", "¡", "&"]

text = ""
with open(file_path, "rt") as file:
    text = file.read()

for (old, new) in replace_subs:
    text = re.sub(old, new, text)

text = text.strip()
doc = nlp(text)

dic = {}
word_count = 0

token_array = []

def add_or_append(dic, key1, key2, value):
    if key1 in dic.keys():
        
        if key2 in dic[key1].keys():
            dic[key1][key2].append(value)
        else:
            dic[key1][key2] = [value]
    else:
        dic[key1] = { key2: [value], }
    return dic

outfile_path = "../data/emos-vs-punks.json"
with open(outfile_path, "w") as outfile:
    outfile.write("""{\n""")
    outfile.write("""  "sentences": [\n""")
    for sentence_i, sentence in enumerate(doc.sents):
        if sentence_i > 0:
            outfile.write(""",\n    [\n""")
        tok_end=","
        for token_i, token in enumerate(sentence):
            if token_i > 0:
                outfile.write(""",\n""")
            pos = "PUNCT" if token.text in punctuation_marks else token.pos_
            
            outfile.write("""      {}\n""".format("{"))
            outfile.write("""        "text":"{}",\n""".format(token.text))
            outfile.write("""        "lemma":"{}",\n""".format(token.lemma_))
            outfile.write("""        "pos":"{}"\n""".format(pos))
            outfile.write("""      {}""".format("}"))
        outfile.write("""\n""")
        outfile.write("""    ]""")
    outfile.write("""\n""")
    outfile.write("""  ]\n""")
    outfile.write("""}""")
            

for sentence_i, sentence in enumerate(doc.sents):
    token_array.append([])
#     print("{: 4d}: {:s}".format(sentence_i, sentence.text.strip()))
    for token_i, token in enumerate(sentence):
        token_array[sentence_i].append(token)
        if token.pos_ == "PUNCT" or token.text in punctuation_marks: # Punctuation
            continue
        word_count += 1
        outer_key = token.lemma_.lower()
        inner_key = token.text.lower()
        dic = add_or_append(dic, outer_key, inner_key, (sentence_i, token_i))
        
#         if key in dic.keys():
#             dic[key].append((sentence_i, token_i))
#         else:
#             dic[key] = [(sentence_i, token_i)]
#         print("\t{:12s} ---> {:12s} ---> {}".format(token.text, token.lemma_, token.pos))
#     if sentence_i >= 10:
#         break
print("Word count: {}".format(word_count))

Word count: 7381


In [5]:
def get_length(inner_dic):
    length = 0
    for key, value in inner_dic.items():
        length += len(value)
    return length

words_counted = 0

for i, (outer_key, inner_dic) in enumerate(sorted(dic.items(), key=lambda item: -get_length(item[1]))):
    inner_dic_len = get_length(inner_dic)
    words_counted += inner_dic_len
    percentage = 100.0 * words_counted / float(word_count)
    print("[{:5.1f}%] {:4d}: \033[1m{:s}\033[22m × {:d}".format(percentage, i + 1, outer_key, inner_dic_len))
    for j, (inner_key, value) in enumerate(sorted(inner_dic.items(), key=lambda item: -len(item[1]))):
        print("\t\t{:3d}: {}".format(j+1, inner_key))
        (sentence_i, token_i) = value[0]
        print("\t\t\t\t", end="")
        tmpstr = ""
        for k, token in enumerate(token_array[sentence_i]):
            if k != 0 and token.pos != 97:
                tmpstr += " "
            if k == token_i:
                tmpstr += "\033[1;4m"
            tmpstr += token.text
            if k == token_i:
                tmpstr += "\033[22;24m"
        print(textwrap.fill(tmpstr, 50).replace("\n", "\n\t\t\t\t"))

[  7.8%]    1: el × 578
		  1: la
				Una advertencia: este episodio contiene
				vocabulario propio de la Ciudad de
				México.
		  2: los
				Para los que no saben o necesitan un
				recordatorio.
		  3: el
				Empecemos el 15 de marzo del 2008,
				en Ciudad de México, que por ese entonces se
				llamaba Distrito Federal.
		  4: las
				La cita era a las tres de la tarde
				en la Glorieta de Insurgentes.
[ 13.5%]    2: de × 421
		  1: de
				Una advertencia: este episodio contiene
				vocabulario propio de la Ciudad de
				México.
[ 17.1%]    3: él × 266
		  1: se
				Empecemos el 15 de marzo del 2008, en Ciudad de
				México, que por ese entonces se
				llamaba Distrito Federal.
		  2: lo
				Lo que están escuchando es un
				reportaje de TV Azteca.
		  3: les
				El cabello largo les caía por la
				frente cubriéndoles los ojos.
		  4: le
				Le pusieron hora, fecha y hasta lema
				al evento: “ Haz patria y mata a un emo ”.
		  5: los
				Y el primer lugar del que los
				quisie

				Esta es Fernanda.
		  3: esto
				Esto es Radio Ambulante desde NPR.
		  4: estos
				Y de repente se veía, yo lo veía con mis propios
				ojos, como una nueva ola de moda principalmente,
				de estos chavalos que se hacían así
				los flecos altísimos de atrás o de enfrente.
		  5: éste
				Éste es Kristoff, un presentador de
				Telehit, uno de esos canales musicales del montón
				que intentaba aspirar a tener un estilo como el de
				MTV.
[ 44.1%]   21: mucho × 50
		  1: muy
				Incluyendo palabras que algunas personas podrían
				considerar ofensivas, pero muy
				chingonas.
		  2: muchos
				Pero igual — y esto sonará raro —
				muchos de los que recordamos esa
				batalla le tenemos algo de cariño.
		  3: mucho
				y sentir mucho.
		  4: mucha
				Ahí andábamos inventándole y. En ese momento no se
				les prestaba mucha atención.
		  5: muchas
				Era una vida más lenta y. de muchas
				formas, más simple.
[ 44.7%]   22: emo × 49
		  1: emo
				Emo viene de emoción.
		  2: e

		  6: poder
				Aunque pisaron por un momento la entrada de la
				tierra prometida, tuvieron que alejarse del Chopo
				para poder terminar el día
				tranquilos.
		  7: podrían
				Incluyendo palabras que algunas personas
				podrían considerar ofensivas, pero
				muy chingonas.
		  8: puedes
				En CarMax puedes hacer tu compra a
				tu manera: en línea desde el confort de tu hogar,
				en persona en el lote, o combinando ambas
				alternativas.
		  9: pueden
				Y pues tal cual nos corrieron, o sea nos corrieron
				así de que: “ Ustedes no pueden
				estar aquí, váyanse”.
		 10: pudo
				necesario crear un nuevo género para expresar
				emociones?,¿ no nos sirve con el death metal?,¿ no
				tenemos suficiente con el punk?,¿ no tenemos
				suficiente con Camila, Sin Bandera, José José?
				Como parte de su investigación, Daniel Hernández
				pudo platicar con él.
		 11: puedo
				Y cerró con esto: Yo puedo no estar
				de acuerdo con lo que piensan los emos o con lo
				que piensa

				Se organizaban para tal día y tal hora salir a
				golpear a los emos.
		  3: golpeaban
				Y al que agarraban lo golpeaban y le
				cortaban el fleco.
		  4: golpeada
				Su hija no fue golpeada en la
				batalla de Insurgentes, pero sí le tocó en otra
				ocasión, mientras caminaba por la calle con sus
				amigos.
[ 62.5%]   86: entonces × 10
		  1: entonces
				Empecemos el 15 de marzo del 2008, en Ciudad de
				México, que por ese entonces se
				llamaba Distrito Federal.
[ 62.7%]   87: joven × 10
		  1: jóvenes
				Jóvenes autodenominados emos estaban
				reunidos para manifestarse y defender su
				ideología.
		  2: joven
				Si fuera un poquito más joven,
				seguramente me hubiera identificado con ellos.
[ 62.8%]   88: cuando × 10
		  1: cuando
				Y cuando estés listo para lanzar tu
				página, usa el código “ NPR ” para ahorrar 10 por
				ciento en la compra de tu primer sitio web o
				dominio.
[ 62.9%]   89: tu × 10
		  1: tu
				Y cuando estés listo para lanzar tu
				

				Luna Negra y Darynkayna les propusieron a los
				chavos emos que organizaran una
				marcha para dar a conocer su postura
				y como un intento de defenderse de manera
				pacífica.
[ 71.8%]  175: desde × 5
		  1: desde
				Esto es Radio Ambulante desde NPR.
[ 71.8%]  176: urbano × 5
		  1: urbanas
				Repasemos: en los 90 se hablaba de tribus
				urbanas, grupos de personas,
				especialmente jóvenes, con gustos similares.
		  2: urbanos
				La reunión comenzó a tomar forma de mitin político
				de grupos urbanos.
[ 71.9%]  177: amenaza × 5
		  1: amenaza
				La amenaza era por parte de grupos
				llamados punketos y así fue.
		  2: amenazas
				Comenzaron los gritos de amenazas,
				de burlas.
[ 72.0%]  178: llamado × 5
		  1: llamados
				La amenaza era por parte de grupos
				llamados punketos y así fue.
		  2: llamadas
				Por incidentes como el de Querétaro y las
				llamadas a la violencia que
				circulaban en internet contra los emos, se sentía
				que las cosas se salían

...

		  1: onda
				pues va a haber tocada,¿ o qué onda?
[ 86.2%]  541: tenso × 2
		  1: tenso
				Trataban de relajarse como cualquier otro sábado,
				pero no podían obviar que el ambiente estaba
				tenso.
[ 86.2%]  542: estár × 2
		  1: estábamos
				Estábamos en este bar que te digo,
				Los Sillones, cuando. cuando ya empezaron a decir
				que sí.
[ 86.3%]  543: resto × 2
		  1: resto
				Los emos que estaban en Los Sillones se levantaron
				y se unieron al resto que estaba
				afuera.
[ 86.3%]  544: acercar él × 2
		  1: acercarse
				Se pusieron a platicar hasta que los interrumpió
				un ruido: de una de las entradas a la Glorieta se
				escuchaban acercarse unos gritos a
				lo lejos.
[ 86.3%]  545: lejos × 2
		  1: lejos
				Se pusieron a platicar hasta que los interrumpió
				un ruido: de una de las entradas a la Glorieta se
				escuchaban acercarse unos gritos a lo
				lejos.
[ 86.3%]  546: ah × 2
		  1: ah
				Y sí le dije: “ ah, no manches, creo
				que sí va haber pedo, gü

[ 89.4%]  707: 2000 × 1
		  1: 2000
				Cuando los emos se pusieron de moda, a mediados de
				los años 2000, yo estaba en la
				secundaria.
[ 89.4%]  708: secundaria × 1
		  1: secundaria
				Cuando los emos se pusieron de moda, a mediados de
				los años 2000, yo estaba en la
				secundaria.
[ 89.4%]  709: década × 1
		  1: década
				Ha pasado más de una década, se
				siente que ha sido una eternidad.
[ 89.4%]  710: eternidad × 1
		  1: eternidad
				Ha pasado más de una década, se siente que ha sido
				una eternidad.
[ 89.4%]  711: información × 1
		  1: información
				Por ejemplo, la información no
				circulaba como hoy.
[ 89.5%]  712: teléfono × 1
		  1: teléfonos
				¿ Recuerdan los teléfonos de ese
				momento?
[ 89.5%]  713: tecla × 1
		  1: teclas
				Con teclas, cámaras de bajísima
				calidad, algunos todavía tenían antenas que tenías
				que sacar para tener señal.
[ 89.5%]  714: bajísimo × 1
		  1: bajísima
				Con teclas, cámaras de bajísima
				calidad, algunos tod

		  1: mercado
				El Chopo es un mercado callejero con
				decenas de puestos de chácharas, ropa, accesorios
				y lo más importante: música, mucha, mucha música.
[ 91.1%]  830: decena × 1
		  1: decenas
				El Chopo es un mercado callejero con
				decenas de puestos de chácharas,
				ropa, accesorios y lo más importante: música,
				mucha, mucha música.
[ 91.1%]  831: cháchara × 1
		  1: chácharas
				El Chopo es un mercado callejero con decenas de
				puestos de chácharas, ropa,
				accesorios y lo más importante: música, mucha,
				mucha música.
[ 91.1%]  832: mucha × 1
		  1: mucha
				El Chopo es un mercado callejero con decenas de
				puestos de chácharas, ropa, accesorios y lo más
				importante: música, mucha, mucha
				música.
[ 91.1%]  833: intercambiar × 1
		  1: intercambian
				No solo se vende, también se
				intercambian álbumes, se organizan
				conciertos y eventos culturales.
[ 91.1%]  834: álbum × 1
		  1: álbumes
				No solo se vende, también se intercambian
				ál

				Y pues, ahora sí que, tal cual así que en el calor
				de lo que pasó, llego y le pego al
				otro y se cae con su novia.
[ 94.2%] 1059: novia × 1
		  1: novia
				Y pues, ahora sí que, tal cual así que en el calor
				de lo que pasó, llego y le pego al otro y se cae
				con su novia.
[ 94.2%] 1060: sensi × 1
		  1: sensi
				verlos a ellos como sensi.
[ 94.2%] 1061: agresivo × 1
		  1: agresivos
				siendo agresivos,¿ no?
[ 94.2%] 1062: chiquillo × 1
		  1: chiquillos
				a todos los chiquillos emos.
[ 94.2%] 1063: dócil × 1
		  1: dóciles
				Y es que a los ojos de todos, los emos eran
				dóciles.
[ 94.2%] 1064: espectador × 1
		  1: espectadores
				Pero bueno, ahí estaban, respondiendo a los
				ataques para sorpresa de los
				espectadores, y claro, de los punks.
[ 94.2%] 1065: volvemo × 1
		  1: volvemos
				Una pausa y volvemos.
[ 94.3%] 1066: hola × 1
		  1: hola
				Hola, soy Paola Aleán, y coordino el
				programa de membresías Deambulantes.
[ 94.3%] 1067: aleán × 1
		  

				Daniel Hernández, que ya tenía bastante tiempo
				dándole seguimiento a los emos,
				también recuerda esa transición.
[ 98.5%] 1379: seguimiento × 1
		  1: seguimiento
				Daniel Hernández, que ya tenía bastante tiempo
				dándole seguimiento a los emos,
				también recuerda esa transición.
[ 98.5%] 1380: transición × 1
		  1: transición
				Daniel Hernández, que ya tenía bastante tiempo
				dándole seguimiento a los emos, también recuerda
				esa transición.
[ 98.5%] 1381: camino × 1
		  1: camino
				Para mí siento que muchos estaban iniciando su
				camino a su eventual cultura con la
				que se iban a quedar.
[ 98.5%] 1382: eventual × 1
		  1: eventual
				Para mí siento que muchos estaban iniciando su
				camino a su eventual cultura con la
				que se iban a quedar.
[ 98.6%] 1383: podcastero × 1
		  1: podcasteros
				podcasteros.
[ 98.6%] 1384: hippie × 1
		  1: hippie
				O hasta un hippie neoazteca.
[ 98.6%] 1385: neoazteco × 1
		  1: neoazteca
				O hasta un hippie neoaz